In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [27]:
import warnings
warnings.filterwarnings('ignore')

## cross_val_score(model, x, Y, scoring, cv, n_jobs, verbose, ... )
- 장점 : 소요 시간, train, test에 대한, 내가 원하는 지표를 다 볼 수 있다
- model : 사용할 모델 ex) decisiontreeclassifier
- scoring : 평가할 스코어링 방법 ex) 'precision', 'accuracy' ...
- cv:  몇번 교차 검증할 것인가?
- n_jobs: ( cpu사용에 대한 지정 -1)
- verbose : 진행 상황 보기

In [28]:
iris_data = load_iris()

data = iris_data.data
label = iris_data.target

In [29]:
dt_clf = DecisionTreeClassifier(random_state=111)

In [30]:
#cross_val_scrore(모델, 데이터, 라벨, 평가지표(scoring), 반복횟수(cv))
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=5) # 5번의 정확도 평가가 되고 리스트 형태로 scores에 저장
print(np.round(np.mean(scores), 3))                                     # mean을 이용하여 5번의 정확도의 평균을 출력

0.96


In [31]:
crs = cross_validate(dt_clf, data, label, scoring='accuracy', cv=5, return_train_score=True)
pd.DataFrame(crs)

,fit_time,score_time,test_score,train_score
0,0.001204,0.000500,0.966667,1.0
1,0.001006,0.000000,0.966667,1.0
2,0.001001,0.001006,0.900000,1.0
3,0.001037,0.000000,0.966667,1.0
4,0.000000,0.001001,1.000000,1.0


In [32]:
scores

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

## 평가지표 - 이진분류 vs 다중분류 

In [33]:
scores = cross_val_score(dt_clf, data, label, scoring='f1', cv=5)
scores

array([nan, nan, nan, nan, nan])

In [34]:
scores = cross_val_score(dt_clf, data, label, scoring='precision', cv=5)
scores

array([nan, nan, nan, nan, nan])

왜 f1, precision에 대한 교차 검증의 값이 NaN인가?

- 이진분류: 0, 1로 나뉘어 있어서 정확도, 정밀도, 재현율, f1스코어 등 0, 1 이진 분류에만 디폴트로 적용된다. 따라서 이진분류는 문제 없이 값들이 나온다   

- 다중분류: 0, 1, 2를 디폴트로 값이 평가되기 때문에 NaN으로 나오게 된다 


## 다중분류 진행 시 정확도 외 다른 평가지표에 대한 가중치 설정 방법

- 다중 분류시 평가지표를 확인하기 위해서는 _micro, _macro, _weighted 이런 식으로 사용하면 된다

- macro : 모든 예측 결과에 대해서 평균을 내어서 값을 계산하는 방법 --> 계산 후 평균

- micro : 각각의 정답에 대한 개수를 가지고 나눠서 평균값을 구한다 --> 데이터를 합쳐서 계산

- weighted : 가중 평균값을 구한다. 가중치를 주고 싶은 것에 줄 수 있다 

In [35]:
# iris 데이터의 경우 label의 값이 0, 1, 2로 이진 분류가 아니다 --> 디폴트로 이진 분류가 적용되는 기존 평가 방식으로는 계산되지 않는다
cross_val_score(dt_clf, data, label, scoring='precision', cv=5)

array([nan, nan, nan, nan, nan])

In [37]:
# ex1) _macro
macro = cross_val_score(dt_clf, data, label, scoring='precision_macro', cv=5)
macro

array([0.96969697, 0.96969697, 0.9023569 , 0.96969697, 1.        ])

In [38]:
# ex2) _micro
micro = cross_val_score(dt_clf, data, label, scoring='precision_micro', cv=5)
micro

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [39]:
# ex3) _weighted
weighted = cross_val_score(dt_clf, data, label, scoring='precision_weighted', cv=5)
weighted

array([0.96969697, 0.96969697, 0.9023569 , 0.96969697, 1.        ])

In [44]:
print(round(np.mean(macro), 3), round(np.mean(micro), 3), round(np.mean(weighted), 3))

0.962 0.96 0.962


하나의 예시로 4개의 라벨(0, 1, 2, 3)이 있다고 가정해보자.   
precision 값을 계산해보니 (0.5, 0.1, 0.5, 0.5)가 순서대로 나왔다고 해보자.   
cf) (1/2, 10/100, 1/2, 1/2)

- macro: (0.5 + 0.1 + 0.5 + 0.5) / 4
- micro: (1 + 10 + 1 + 1) / (2 + 100 + 2 + 2)

=> 그렇다면 이 두 가지는 언제 사용해야 하는가? 둘의 차이는 무엇인가??   
Ans) 클래스에 대한 불균형이 있을 때(같은 비율이더라도 분모가 큰 경우가 있으니) micro는 이를 반영한다

예를 들어, (0, 1, 2, 3)의 라벨에 대해 (2, 100, 2, 2)의 데이터가 있다면,   
랜덤하게 데이터를 뽑을 때 확률적으로 1을 뽑을 확률이 높은 것은 당연하다   
=> 데이터의 불균형이 있다

## LeaveOneOut 교차 검증



In [45]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=loo)

print(len(scores))  # 데이터의 수만큼
print(scores.mean())

150
0.94


In [47]:
import seaborn as sns
dt_clf = DecisionTreeClassifier(random_state=111)
df = sns.load_dataset('titanic')
df_tt = df[['pclass', 'fare']]

## Group교차검증

- 그룹핑은 그룹핑 데이터를 가지고 교차검증할 때 더 좋은 성능을 보여줄 수 있다.
- 단순하게 교차검증을 cross 하는 게 아니라
- 데이터의 특성을 보고 내가 분석  할 것이 어떤 클래스별의 수치를 비교하는 것이라면 교차검증 자체도 그룹핑으로 접근할 수 있는 것

In [51]:
## cross val 응용하기 
model = [DecisionTreeClassifier(),RandomForestClassifier(),LogisticRegression()] 
name = ['DT','RF','LR']

for model ,name in zip(model, name):
    print('******************사용한 알고리즘',name,'******************')
    for score in ['accuracy','precision','recall','f1']:
        print(score, round(np.mean(cross_val_score(model, df_tt, df['survived'], scoring=score,cv=10)),3))

******************사용한 알고리즘 DT ******************
accuracy 0.686
precision 0.617
recall 0.465
f1 0.527
******************사용한 알고리즘 RF ******************
accuracy 0.701
precision 0.636
recall 0.535
f1 0.573
******************사용한 알고리즘 LR ******************
accuracy 0.679
precision 0.63
recall 0.398
f1 0.483


## 평가지표

- 분류
- 회귀

### 분류

- 정확도, 정밀도, 재현율, f1, roc
- 구성 요소가 어떤 식으로 되어 있는 것인가
- 정밀도와 재현율의 차이가 어떤 상황에 맞는 것인가
- 왜 그래서 f1 스코어를 보는 것인가
- roc 커브에 x축과 y축이 의미하는 것은 무엇인

- 정확도(Accuracy) : 맞춘 예측 수 / 전체 예측 데이터 건수
cf) 이진 분류(0, 1)이 default이고 0을 0으로 맞추고, 1을 1로 맞춘 것이 모두 맞춘 수에 포함   
   

```
            - 예측값 -
         0             1

    0    TN            FP
실       
제   
값   
    1    FN            TP
```

- TN: True Negative     - 0으로 맞춘 경우(예측 성공)
- FP: False Positive    - 1로 맞추지 못한 경우(예측 실패)
- FN: False Negative    - 0으로 맞추지 못한 경우(예측 실패)
- TP: True Positive     - 1로 맞춘 경우(예측 성공)